In [1]:
#Activation of the Python libraries for numerical and graphical analyses
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt


KeyboardInterrupt



In [ ]:
#Freezing sampling order and importing the dataset 
seed_value= 400 #random seed number
import random 
random.seed(seed_value)
bp = pd.read_excel('Database source') #specify path of your database file (INFILLED FRAMES)

In [ ]:
bp.describe() #important informations about the distributions of the dataset variables

In [ ]:
#Dataset train/test splitting strategy
from sklearn.model_selection import train_test_split 

In [ ]:
X = bp[['I','II','III','IV','V']].values #input samples (INFILLED frames)

In [ ]:
y = bp['VI'].values #Output samples (INFILLED frames)

In [ ]:
np.random.seed(seed_value) #freezing

In [ ]:
#Data preprocessing (normalization)
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaling1 = MinMaxScaler(feature_range=(-1, 1)) #range between -1 and 1 for input data
scaling2 = MinMaxScaler(feature_range=(-1, 1)) #range between -1 and 1 for output data

In [ ]:
#Activation of TensorFlow and Sklearn libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import models
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',mode='min',patience=100, restore_best_weights=True) #Implementation of early stopping rule

In [ ]:
tf.random.set_seed(seed_value) #Freezing the results
kv = KFold(n_splits=10, shuffle=True, random_state=seed_value) #defining the CV procedure
cv1_r2_scores_TEST=[] #R^2 scores on the test set
cv1_r2_scores_TRAIN=[] #R^2 scores on the training set
MSE1_scores_TEST=[] #mean squared error scores
RMSE1_scores_TEST=[] #root mean squared error scores

In [ ]:
#Hyperparameters
import tensorflow.keras as k
from sklearn import metrics
y=np.reshape(y,(2178,1))

for train, test in kv.split(X, y):
    scaling1.fit(X[train])
    X[train] = scaling1.transform(X[train])
    X[test] = scaling1.transform(X[test])
    scaling2.fit(y[train])
    y[train] = scaling2.transform(y[train])
    y[test] = scaling2.transform(y[test])
    
    #ANN architecture
    model=Sequential()
    model.add(Dense(10,activation='tanh')) 
    model.add(Dense(1,activation='linear')) 
    
    #ANN algorithm/optimizer - Adam optimizer
    opt1 = tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.9,epsilon=1e-08) #Adam or SGD optimizer - tuning hyperparameters
    model.compile(optimizer=opt1,loss='mse',steps_per_execution=5,run_eagerly=False)
    model.fit(X[train], y[train], epochs=1000,validation_data=(X[test], y[test]),shuffle=False,callbacks=[early_stopping])
    
    #ANN performance metrics
    test_prediction = model.predict(X[test])
    trening_prediction = model.predict(X[train])
    
    R_square_TEST1= metrics.r2_score(y[test],test_predikcija)
    MSE_TEST1= metrics.mean_squared_error(y[test],test_predikcija)
    RMSE_TEST1= metrics.mean_squared_error(y[test],test_predikcija)**0.5
    R_square_TRAIN1=metrics.r2_score(y[train],trening_predikcija)
    
    #displaying numerical results
    print ('Coefficient of determination R2-TEST', R_square_TEST1)
    print ('Coefficient of determination R2-TRAIN', R_square_TRAIN1)
    print ('MSE-TEST', MSE_TEST1)
    print ('RMSE-TEST', RMSE_TEST1)
    
    #set of all results
    cv1_r2_scores_TEST.append(R_square_TEST1)
    cv1_r2_scores_TRAIN.append(R_square_TRAIN1)
    MSE1_scores_TEST.append(MSE_TEST1)
    RMSE1_scores_TEST.append(RMSE_TEST1)
    
#mean values of the performance indicators
print(np.mean(cv1_r2_scores_TEST)) 
print(np.hstack(cv1_r2_scores_TEST))
print(np.mean(cv1_r2_scores_TRAIN))
print(np.hstack(cv1_r2_scores_TRAIN))
print(np.mean(MSE1_scores_TEST))
print(np.mean(RMSE1_scores_TEST))

In [ ]:
#Graphical presentation of the results of the coefficients of determination - Test results
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
threshold = np.mean(cv1_r2_scores_TEST)
ax.set_yticks(np.arange(0.9, 1.01, 0.02))
plt.ylim(0.9,1.01)
ax.set_xlabel('Folds',fontsize=16)
ax.set_ylabel('R²',fontsize=16)
nums = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
rsquared = np.hstack(cv1_r2_scores_TEST)
plt.axhline(threshold, color='red', ls='dotted', lw=3)
ax.bar(nums,rsquared,width=0.5)
mn=str("{:.3f}".format(np.mean(cv1_r2_scores_TEST)))
plt.text(3,0.99,'Mean='+ mn,fontsize=16)
plt.show()
fig.savefig('Figure name 1.svg',dpi=1800) #set title of the figure

In [ ]:
#Graphical presentation of the results of the coefficients of determination - Train results
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
threshold = np.mean(cv1_r2_scores_TRAIN)
ax.set_yticks(np.arange(0.9, 1.01, 0.02))
plt.ylim(0.9,1.01)
ax.set_xlabel('Folds',fontsize=16)
ax.set_ylabel('R²',fontsize=16)
nums = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
rsquared = np.hstack(cv1_r2_scores_TRAIN)
plt.axhline(threshold, color='red', ls='dotted', lw=3)
ax.bar(nums,rsquared,width=0.5)
mn=str("{:.3f}".format(np.mean(cv1_r2_scores_TRAIN)))
plt.text(3,0.99,'Mean='+ mn,fontsize=16)
plt.show()
fig.savefig('Figure name 2.svg',dpi=1800) #set title of the figure